## Main Code

In [1]:
import mysql.connector
import time

def run_query_and_measure_time(
    host="localhost", #192.168.31.37
    database="computer_shop",
    user="root",
    password="1234",
    query='',
):
    start_time = time.time()

    try:
        conn = mysql.connector.connect(
            host=host,
            database=database,
            user=user,
            password=password
        )
        cursor = conn.cursor()

        cursor.execute(query)
        rows = cursor.fetchall()

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        rows = []

    finally:
            cursor.close() # type: ignore
            conn.close() # type: ignore

    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Time taken: {elapsed_time} seconds - {len(rows)} rows")
    return elapsed_time

In [2]:
#1
run_query_and_measure_time(query='''
SELECT
    c.customer_id,
    c.name AS customer_name,
    c.phone_number,
    COUNT(o.order_id) AS total_orders,
    SUM(pt.amount) AS total_spent
FROM
    customer c
JOIN
    orders o ON c.customer_id = o.customer_id
LEFT JOIN
    paymenttransaction pt ON o.order_id = pt.order_id
GROUP BY
    c.customer_id, c.name, c.phone_number
ORDER BY
    total_spent DESC;
''')

Time taken: 165.68152832984924 seconds - 140000 rows


165.68152832984924

In [3]:
#2
run_query_and_measure_time(query='''
SELECT
    p.product_code,
    p.name AS product_name,
    b.name AS brand_name,
    c.title AS category_title,
    AVG(pf.rating) AS average_rating,
    COUNT(pf.feedback_id) AS feedback_count
FROM
    product p
JOIN
    brand b ON p.brand_id = b.id
JOIN
    category c ON p.category_id = c.category_id
JOIN
    productfeedback pf ON p.product_code = pf.product_code
GROUP BY
    p.product_code, p.name, b.name, c.title
HAVING
    AVG(pf.rating) >= 4
ORDER BY
    average_rating DESC;
''')

Time taken: 2.4608728885650635 seconds - 1700 rows


2.4608728885650635

In [ ]:
#3
run_query_and_measure_time(query='''
SELECT 
    s.staff_id,
    s.name AS staff_name,
    s.position,
    COUNT(DISTINCT il.log_id) AS inventory_updates,
    COALESCE(SUM(oi.qty * oi.price_at_purchase), 0) AS total_order_value,
    COUNT(DISTINCT o.order_id) AS total_orders_handled
FROM staff s
LEFT JOIN inventorylog il ON s.staff_id = il.staff_id
LEFT JOIN orderitem oi ON il.product_code = oi.product_code
LEFT JOIN orders o ON oi.order_id = o.order_id AND o.order_status IN ('Delivered', 'Processing')
WHERE s.is_active = TRUE
GROUP BY s.staff_id, s.name, s.position
HAVING total_orders_handled > 0;

''')

Time taken: 0.024944067001342773 seconds - 0 rows


0.024944067001342773

In [5]:
#4
run_query_and_measure_time(query='''
SELECT 
    pm.pay_method_id,
    pm.pay_method,
    pm.company_handle,
    COUNT(pt.transaction_id) AS transaction_count,
    COALESCE(SUM(pt.amount), 0) AS total_amount,
    COUNT(DISTINCT CASE WHEN pt.status = 'Completed' THEN pt.transaction_id END) AS completed_transactions
FROM paymentmethod pm
LEFT JOIN paymenttransaction pt ON pm.pay_method_id = pt.pay_method_id
WHERE pm.is_enable = TRUE
GROUP BY pm.pay_method_id, pm.pay_method, pm.company_handle
ORDER BY total_amount DESC;
''')

Time taken: 24.201350450515747 seconds - 5 rows


24.201350450515747

In [4]:
#5
run_query_and_measure_time(query='''
SELECT 
    p.promotion_id,
    p.title AS promotion_title,
    p.discount_type,
    p.discount_value,
    COUNT(DISTINCT pp.product_code) AS products_affected,
    COALESCE(SUM(oi.qty * oi.price_at_purchase), 0) AS total_sales_under_promotion
FROM promotion p
LEFT JOIN productpromotion pp ON p.promotion_id = pp.promotion_id
LEFT JOIN orderitem oi ON pp.product_code = oi.product_code
LEFT JOIN orders o ON oi.order_id = o.order_id AND o.order_status = 'Delivered'
WHERE p.start_date <= CURRENT_TIMESTAMP AND p.end_date >= CURRENT_TIMESTAMP
GROUP BY p.promotion_id, p.title, p.discount_type, p.discount_value;
''')

Time taken: 0.02167677879333496 seconds - 0 rows


0.02167677879333496

In [3]:
#6
run_query_and_measure_time(query='''
SELECT 
    p.product_code,
    p.name AS product_name,
    AVG(pf.rating) AS average_rating,
    COUNT(pf.feedback_id) AS feedback_count,
    COUNT(DISTINCT pf.customer_id) AS unique_customers
FROM product p
LEFT JOIN productfeedback pf ON p.product_code = pf.product_code
GROUP BY p.product_code, p.name
HAVING feedback_count > 0
ORDER BY average_rating DESC;
''')

Time taken: 4.595085382461548 seconds - 15883 rows


4.595085382461548

In [3]:
#7
run_query_and_measure_time(query='''
SELECT
    cat.category_id,
    cat.title AS category_name,
    SUM(oi.qty * oi.price_at_purchase) AS total_category_revenue
FROM
    orderitem oi
JOIN
    product p ON oi.product_code = p.product_code
JOIN
    category cat ON p.category_id = cat.category_id
GROUP BY
    cat.category_id, cat.title
ORDER BY
    total_category_revenue DESC;
''')

Time taken: 1973.160965681076 seconds - 7 rows


1973.160965681076

In [ ]:
#8
run_query_and_measure_time(query='''
SELECT
    o.order_id,
    o.order_date,
    o.order_status,
    c.customer_id,
    c.name AS customer_name,
    c.phone_number AS customer_phone,
    a.street_line,
    a.commune,
    a.district,
    a.province
FROM
    orders o
JOIN
    customer c ON o.customer_id = c.customer_id
JOIN
    address a ON o.address_id = a.address_id
WHERE
    o.order_status = 'Pending';
''')

In [ ]:
#10
run_query_and_measure_time(query='''
SELECT
    p.product_code,
    p.name AS product_name,
    p.stock_quantity,
    p.last_restock_date,
    s.name AS supplier_name -- Adding supplier name for more context
FROM
    product p
LEFT JOIN
    productsupplier ps ON p.product_code = ps.product_code
LEFT JOIN
    supplier s ON ps.supplier_id = s.supplier_id
WHERE
    p.stock_quantity < 10
ORDER BY
    p.stock_quantity ASC, p.last_restock_date DESC;
''')

In [ ]:
#9
run_query_and_measure_time(query='''
SELECT
    p.name,
    p.product_code,
    TotalQuantitySold
FROM
    product p
JOIN
    (SELECT
        product_code,
        SUM(qty) AS TotalQuantitySold
    FROM
        orderitem
    GROUP BY
        product_code
    ) AS Sales
ON
    p.product_code = Sales.product_code
ORDER BY
    TotalQuantitySold DESC
LIMIT 5;
''')

Time taken: 1951.4089477062225 seconds - 5 rows


1951.4089477062225